In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from sklearn.metrics import f1_score
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split



pd.options.mode.chained_assignment = None

In [2]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), 
#           " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(
#         gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [3]:
torch.cuda.empty_cache()
#printm()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!ls '/content/drive/My Drive/app'

In [5]:
mimic_data = pd.read_csv("/content/drive/My Drive/app/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
mimic_data = mimic_data.loc[mimic_data['No Finding'] == 0]
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,50629293,16059088,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,50113713,16059088,b' WET READ: ___ ___ ___ 7:27 PM\n Left PICC ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,57620089,16059088,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
binary_data = mimic_data[['text','Lung Lesion']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,Lung Lesion
0,"b"" final repor...",0.0
1,b' final repor...,0.0
5,"b"" final repor...",0.0
6,b' wet read: ___ ___ ___ 7:27 pm\n left picc ...,0.0
7,b' final repor...,0.0


In [8]:
binary_data.groupby('Lung Lesion').count()

,text
Lung Lesion,
0.0,146088
1.0,6284


In [9]:
negative = binary_data.loc[binary_data['Lung Lesion']==0].sample(n=20000)
print(len(negative))
positive = binary_data.loc[binary_data['Lung Lesion']==1].sample(n=len(negative),replace=True)

20000


In [10]:
binary_data = positive.append(negative,ignore_index=True).reset_index(drop=True)
#df1.append(df2, ignore_index=True)

In [11]:
len(binary_data)

40000

In [12]:
binary_data.text[0]

"b'                                 final report\\n history:  chf.\\n \\n findings:  in comparison with the study of ___, the monitoring and support\\n devices are in unchanged position.  the patient has taken a slightly better\\n inspiration.  nevertheless, there is continued enlargement of the cardiac\\n silhouette with relatively mild pulmonary vascular congestion.  the\\n possibility of cardiomyopathy or pericardial effusion would have to be\\n considered.  hazy opacification at the bases could reflect small layering\\n effusions with compressive atelectasis.\\n \\n patchy area of increased opacification in the right upper zone is consistent\\n with mass with calcification seen on prior ct study.\\n'"

In [13]:
y = binary_data['Lung Lesion']
X = binary_data.drop(columns=['Lung Lesion'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [14]:
X_train.head()

,text
0,b' final repor...
1,b' final repor...
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [15]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [16]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0])))

'b final report history chf findings in comparison with the study of the monitoring and support devices are in unchanged position the patient has taken a slightly better inspiration nevertheless there is continued enlargement of the cardiac silhouette with relatively mild pulmonary vascular congestion the possibility of cardiomyopathy or pericardial effusion would have to be considered hazy opacification at the bases could reflect small layering effusions with compressive atelectasis patchy area of increased opacification in the right upper zone is consistent with mass with calcification seen on prior ct study'

In [17]:
#spacy_tok(binary_data.text[0])[1:]

In [18]:
binary_train = binary_data.copy()
binary_train.head()

,text,Lung Lesion
0,b' final repor...,1.0
1,b' wet read: ___ ___ ___ 7:56 pm\n no acute c...,1.0
2,b' final repor...,1.0
3,b' final repor...,1.0
4,b' wet read: ___ ___ ___ 7:25 pm\n no radiogr...,1.0


In [19]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word)[1:])
    return counts

In [20]:
word_count = get_counts(binary_train.text)

In [21]:
len(word_count.keys())

14865

In [22]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [23]:
len(word_count.keys())

8711

In [24]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [25]:
len(words)

8713

In [26]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [27]:
encode_sentence(binary_train.text[0])

array([ 1,  2,  1,  1,  1,  1,  1,  7,  8,  9, 10, 11, 12,  1, 10, 13, 14,
        1, 16, 17,  7, 18,  1, 10, 20, 21, 22, 23, 24,  1,  1,  1, 28, 29,
       30, 31, 12, 10,  1, 33,  9, 34, 35, 36, 37,  1,  1, 39, 12, 40, 41,
       42, 43, 44, 45, 46,  1,  1, 49, 50, 51, 10, 52, 53, 54, 55,  1, 57,
        9, 58,  1,  1, 60, 61, 12, 62, 50,  7, 10, 63, 64, 65, 29,  1,  9,
       67,  9, 68, 69, 70, 71, 72,  1])

## Dataset and data loaders

In [28]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [29]:
b_train = Binary_Mimic(X_train, y_train, vocab2index)
b_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [30]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [31]:
# data = [b_train[0], b_train[1], b_train[2]]
# data

In [32]:
# collate_fn(data)

In [33]:
# train_loader = DataLoader(b_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
# sents, lengths, labels = next(iter(train_loader))
# sents

In [34]:
# sents.shape, labels.shape

In [35]:
# lengths

## Training

In [36]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.cuda().long()
            y = y.cuda().float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))
  

In [37]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    with torch.no_grad():
      for x, s, y in valid_dl:
          x = x.cuda().long()
          y = y.cuda().float().unsqueeze(1)
          y_hat = model(x, s)
          loss = F.binary_cross_entropy_with_logits(y_hat, y)
          y_pred = y_hat > 0
          correct += (y_pred.float() == y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [38]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        #print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [39]:
torch.cuda.empty_cache()
#printm()

In [40]:
batch_size = 25000
train_dl = DataLoader(b_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(b_valid, batch_size=batch_size, collate_fn=collate_fn)

In [41]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

8713


In [42]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.709 val loss 0.697 and val accuracy 0.495
Epoch #2: train loss 0.692 val loss 0.693 and val accuracy 0.509
Epoch #3: train loss 0.694 val loss 0.679 and val accuracy 0.548
Epoch #4: train loss 0.678 val loss 0.663 and val accuracy 0.612
Epoch #5: train loss 0.661 val loss 0.654 and val accuracy 0.622
Epoch #6: train loss 0.647 val loss 0.618 and val accuracy 0.660
Epoch #7: train loss 0.612 val loss 0.556 and val accuracy 0.718
Epoch #8: train loss 0.558 val loss 0.735 and val accuracy 0.651
Epoch #9: train loss 0.617 val loss 0.482 and val accuracy 0.789
Epoch #10: train loss 0.510 val loss 0.478 and val accuracy 0.792
Epoch #11: train loss 0.498 val loss 0.489 and val accuracy 0.779
Epoch #12: train loss 0.499 val loss 0.465 and val accuracy 0.794
Epoch #13: train loss 0.471 val loss 0.386 and val accuracy 0.848
Epoch #14: train loss 0.409 val loss 0.333 and val accuracy 0.883
Epoch #15: train loss 0.372 val loss 0.363 and val accuracy 0.863


## Use Glove to create pre-trained embedding 

In [43]:
def loadGloveModel(gloveFile='/content/drive/My Drive/app/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [44]:
word_vecs = loadGloveModel()

In [45]:
word_count = get_counts(binary_train.text)

In [46]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 14865


In [47]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [48]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

11341


In [49]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [50]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [51]:
len(pretrained_weight)

11343

In [52]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0698,  0.1673,  0.2341,  ...,  0.1459,  0.0261, -0.1492],
        [-0.4124,  0.6493, -0.5585,  ...,  0.2621,  0.1045, -0.4430],
        ...,
        [ 0.7515, -0.0905,  0.5094,  ..., -0.4257, -0.0778, -0.3276],
        [ 0.0621, -0.4210, -0.4742,  ...,  1.0469,  0.0970, -0.4673],
        [ 0.3502, -0.3619,  1.5050,  ..., -0.3152,  0.9638, -0.5512]])

# GRU Model with Pretrained embedding layer:

In [53]:
class GRUModel_pre(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel_pre,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [54]:
torch.cuda.empty_cache()
#printm()

In [55]:
vocab_size = len(pretrained_weight)
model = GRUModel_pre(vocab_size, 50, 50, glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [56]:
train_epocs(model, optimizer,train_dl, valid_dl, epochs=5)

Epoch #1: train loss 0.695 val loss 0.683 and val accuracy 0.552
Epoch #2: train loss 0.685 val loss 0.666 and val accuracy 0.604
Epoch #3: train loss 0.675 val loss 0.646 and val accuracy 0.628
Epoch #4: train loss 0.662 val loss 0.635 and val accuracy 0.636
Epoch #5: train loss 0.652 val loss 0.614 and val accuracy 0.662


In [57]:
model.embeddings.weight.requires_grad = True

In [58]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.688 val loss 0.592 and val accuracy 0.689
Epoch #2: train loss 0.610 val loss 0.611 and val accuracy 0.673
Epoch #3: train loss 0.631 val loss 0.620 and val accuracy 0.694
Epoch #4: train loss 0.636 val loss 0.611 and val accuracy 0.702
Epoch #5: train loss 0.627 val loss 0.579 and val accuracy 0.720
Epoch #6: train loss 0.599 val loss 0.570 and val accuracy 0.710
Epoch #7: train loss 0.567 val loss 0.492 and val accuracy 0.778
Epoch #8: train loss 0.507 val loss 0.486 and val accuracy 0.791
Epoch #9: train loss 0.458 val loss 0.426 and val accuracy 0.829
Epoch #10: train loss 0.425 val loss 0.391 and val accuracy 0.847
Epoch #11: train loss 0.382 val loss 0.321 and val accuracy 0.880
Epoch #12: train loss 0.355 val loss 0.320 and val accuracy 0.876
Epoch #13: train loss 0.316 val loss 0.293 and val accuracy 0.887
Epoch #14: train loss 0.294 val loss 0.283 and val accuracy 0.894
Epoch #15: train loss 0.264 val loss 0.254 and val accuracy 0.910
